# Различия в активности пользователей Яндекс Книги: Москва и Санкт-Петербург

- Автор: Пухова Евгения
- Дата: 15.03.2025

## Цели и задачи проекта

**Цель проекта:**

Анализ пользовательской активности в сервисе Яндекс Книги для выявления закономерностей поведения пользователей, оценки ключевых метрик и проверки гипотез о различиях в активности между пользователями из Москвы и Санкт-Петербурга.

**Основные задачи:**

1. Проверить гипотезу о разнице в средней активности пользователей из двух городов (Москва и Санкт-Петербург):

- Загрузить и предобработать данные
- Сравнить размеры групп, их статистики и распределения
- Провести односторонний статистический тест на значимость различий

2. Подготовить аналитическую записку, включающую:

- Выводы по метрикам
- Результаты проверки гипотезы
- Интерпретацию значимости различий
- Рекомендации для бизнеса на основе полученных данных

## Описание данных

В этом проекте анализируются данные о чтении и прослушивании контента в сервисе Яндекс Книги. Данные охватывают период с 1 сентября по 11 декабря 2024 года и включают информацию о пользователях, платформах, длительности сессий и местоположении.

Используемые таблицы:

`bookmate.audition` — данные об активности пользователей:
`puid` — идентификатор пользователя
`msk_business_dt_str` — дата события
`hours` — количество часов активности
`usage_geo_id` — идентификатор геопозиции
`bookmate.geo` — данные о местоположении:
`usage_geo_id_name` — город пользователя

Файл `yandex_knigi_data.csv` содержит агрегированные данные:
`city` — город пользователя (Москва / Санкт-Петербург)
`puid` — уникальный идентификатор пользователя
`hours` — общее количество часов активности

## Содержимое проекта

1. Загрузка данных и знакомство с ними
2. Проверка дубликатов и работа с ними
3. Проверка гипотезы о разнице в активности пользователей Москвы и Санкт-Петербурга:
- H₀ (нулевая гипотеза): среднее время активности пользователей не различается между городами.
- H₁ (альтернативная гипотеза): пользователи из Санкт-Петербурга проводят больше времени за чтением и прослушиванием.
4. Проведение t-теста для независимых выборок
5. Интерпретация результатов теста
6. Аналитическая записка
- Выводы по результатам теста
- Возможные объяснения полученных результатов

## Загрузка данных и знакомство с ними

Загрузите данные пользователей из Москвы и Санкт-Петербурга c их активностью (суммой часов чтения и прослушивания) из файла `/datasets/yandex_knigi_data.csv`.

In [1]:
import pandas as pd
df = pd.read_csv('https://code.s3.yandex.net/datasets/yandex_knigi_data.csv')
df.head()

,Unnamed: 0,city,puid,hours
0,0,Москва,9668,26.167776
1,1,Москва,16598,82.111217
2,2,Москва,80401,4.656906
3,3,Москва,140205,1.840556
4,4,Москва,248755,151.326434


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  8784 non-null   int64  
 1   city        8784 non-null   object 
 2   puid        8784 non-null   int64  
 3   hours       8784 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 274.6+ KB


Данные загружены корректно, пропущенных значений нет, типы данных совпадают с ожидаемым

In [3]:
# Проверим дубликаты
duplicates = df.duplicated().sum()
print(f"Количество полных дубликатов: {duplicates}")

# Поиск дубликатов по идентификатору пользователя
duplicate_puid = df.duplicated(subset=['puid']).sum()
print(f"Количество повторяющихся puid: {duplicate_puid}")

Количество полных дубликатов: 0
Количество повторяющихся puid: 244


In [4]:
df[df.duplicated(subset=['puid'], keep=False)].sort_values(by='puid').head(10)

,Unnamed: 0,city,puid,hours
35,35,Москва,2637041,10.317371
6247,6247,Санкт-Петербург,2637041,3.883926
134,134,Москва,9979490,32.415573
6274,6274,Санкт-Петербург,9979490,1.302997
145,145,Москва,10597984,42.931506
6279,6279,Санкт-Петербург,10597984,9.041320
150,150,Москва,10815097,9.086655
6283,6283,Санкт-Петербург,10815097,0.323291
187,187,Москва,13626259,21.104167
6300,6300,Санкт-Петербург,13626259,1.648434


Мы видим, что один и тот же пользователь был в двух городах, какие могут быть причины:

Пользователь мог менять местоположение - например, сначала был активен в Москве, потом в Санкт-Петербурге
Ошибка в данных - возможно, система некорректно присваивала локацию (например, если пользователь использовал VPN)

In [5]:
df = df.sort_values(by=['puid', 'hours'], ascending=[True, False]).drop_duplicates(subset=['puid'], keep='first')
duplicate_puid = df.duplicated(subset=['puid']).sum()
print(f"Количество повторяющихся puid после обработки: {duplicate_puid}")

Количество повторяющихся puid после обработки: 0


Гипотеза сравнивает среднее время активности пользователей между двумя городами, а если пользователь появляется в обеих группах, он искажает распределение. Чтобы правильно сравнить две независимые выборки, важно, чтобы каждый puid был только в одном городе.

Оставим только один город для каждого puid, с наибольшим временем, т.к. если человек больше времени слушал/читал в Питере, значит, он более активен именно там.

Это даст чистое сравнение средних значений между Москвой и Санкт-Петербургом.

## Проверка гипотезы в Python

Гипотеза звучит так: пользователи из Санкт-Петербурга проводят в среднем больше времени за чтением и прослушиванием книг в приложении, чем пользователи из Москвы. Попробуйте статистически это доказать, используя одностороннюю проверку гипотезы с двумя выборками:

- Нулевая гипотеза H₀: Средняя активность пользователей в часах в двух группах (Москва и Санкт-Петербург) не различается.

- Альтернативная гипотеза H₁: Средняя активность пользователей в Санкт-Петербурге больше, и это различие статистически значимо.

In [6]:
import scipy.stats as stats

# Разделим данные на две группы
moscow_hours = df[df['city'] == 'Москва']['hours']
spb_hours = df[df['city'] == 'Санкт-Петербург']['hours']

alpha = 0.05

# Проверим равенство дисперсий
levene_stat, levene_p = stats.levene(moscow_hours, spb_hours)
equal_var = levene_p > alpha  # Если p > alpha, считаем дисперсии равными

# Односторонний t-тест
t_stat, p_value = stats.ttest_ind(spb_hours, moscow_hours, alternative='greater', equal_var=equal_var)

print(f"Levene test (проверка дисперсий): p-value = {levene_p:.5f}")
print(f"T-test: t-статистика = {t_stat:.5f}, p-value = {p_value:.5f}")

# Интерпретация результата
if p_value < alpha:
    print("Отвергаем нулевую гипотезу: пользователи из Санкт-Петербурга проводят больше времени за чтением и прослушиванием.")
else:
    print("Не отвергаем нулевую гипотезу: статистически значимых различий нет.")

Levene test (проверка дисперсий): p-value = 0.30882
T-test: t-статистика = 1.04186, p-value = 0.14875
Не отвергаем нулевую гипотезу: статистически значимых различий нет.


**Выводы по проверке гипотезы:**

- Так как p-value > 0.05, различий в дисперсиях нет, то можно считать дисперсии равными (equal_var=True в t-тесте).

- p-value > 0.05, поэтому мы не можем отвергнуть нулевую гипотезу, это означает, что нет статистически значимых доказательств того, что пользователи из Санкт-Петербурга проводят в среднем больше времени за чтением и прослушиванием, чем пользователи из Москвы

## Аналитическая записка

1. Выбранный метод тестирования
- Для проверки гипотезы использован t-тест для независимых выборок, т.к у нас две независимые группы пользователей (Москва и Санкт-Петербург)
- Проводился односторонний тест, т.к гипотеза предполагала, что пользователи из Санкт-Петербурга проводят больше времени за чтением и прослушиванием контента, чем пользователи из Москвы.
- Уровень статистической значимости (α) = 5%, что является стандартным критерием принятия решений в статистическом анализе.

2. Результаты t-теста
- t-статистика = 1.04186
- p-value = 0.14875

3. Вывод
Так как p-value (0.14875) > 0.05, мы не отвергаем нулевую гипотезу.
Это означает, что нет статистически значимых доказательств того, что пользователи из Санкт-Петербурга проводят в среднем больше времени за чтением и прослушиванием книг, чем пользователи из Москвы.

4. Возможные объяснения полученных результатов
- LTV выше в Санкт-Петербурге по другим причинам (например, в Санкт-Петербурге выше доля пользователей с долгосрочной подпиской или в Москве больше пользователей, использующих другие сервисы Яндекс Плюс, что может размывать связь между временем активности и LTV)
- Разница в пользовательских привычках, например, московские пользователи могут чаще читать короткие произведения или слушать аудиокниги в дороге, но проводить в сумме столько же времени, сколько и пользователи в Санкт-Петербурге.